# Exampe: sphere with PS3 dataset

This notebook shows how to use `ffsas` to invert for the radius distribution of a `Sphere` model with a real dataset called "PS3". It uses the [SASView/SASModels](http://www.sasview.org/docs/user/models/sphere.html) unit system.

In [ ]:
# avoid omp error on Mac
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# plotting setup
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'legend.fontsize': 11})
matplotlib.rcParams.update({'axes.titlesize': 12})
matplotlib.rcParams.update({'lines.linewidth': 2})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# create output dir
from pathlib import Path
output_dir = Path('./output/sphere_PS3_data')
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# uncomment this line to install ffsas
# !pip install ffsas

# ffsas
import torch
import ffsas
from ffsas.models import Sphere
from ffsas.system import SASGreensSystem

# math tools
from scipy import interpolate

# numpy for reading data
import numpy as np

# Read data

Data are stored in the text file `PS3_data/observation.txt`, with the three columns being $q$, mean and standard deviation of the observed intensity.

In [ ]:
# read data
fname = f'PS3_data/observation.txt'
data = np.loadtxt(fname)

# q vector
q = torch.tensor(data[:, 0], dtype=ffsas.torch_dtype)

# intensity mean
mu = torch.tensor(data[:, 1], dtype=ffsas.torch_dtype)

# intensity stddev
sigma = torch.tensor(data[:, 2], dtype=ffsas.torch_dtype)

# Inversion

Just a few lines to do inversion:

In [ ]:
# specify radii
r = torch.linspace(350, 850, 500)

# compute the Green's tensor
G = Sphere.compute_G_mini_batch([q], {'r': r}, {'drho': 1.})

# build G-system
g_sys = SASGreensSystem(G, Sphere.get_par_keys_G(), log_file=output_dir / 'sphere_PS3_data.log')

# inversion
# do only 40 iterations and save every 10 iterations
results = g_sys.solve_inverse(mu, sigma, maxiter=40, save_iter=10)

# Plot results

We also plot the results from `MCSAS` and `SASView` for comparison.

In [ ]:
# line stype, width, colors
dashes = [[1, 0], [3, 1.5], [2, 1], [1, 1], [.5, .5]]
lw = [1.5, 1.5, 1.5, 1.5, 1.5]

# saved steps to be plotted
plot_steps = [1, 2]

# intensity
plt.figure(dpi=200, figsize=(8, 4))
plt.errorbar(q, mu, yerr=sigma, c='pink', ecolor='skyblue', lw=1, fmt='o',
             markersize=3, label=r'Observed $I(q)$', zorder=-100)
for j, step in enumerate(plot_steps):
    I = results['saved_res'][step]['I']
    chi2 = np.linalg.norm((mu - I) / sigma) ** 2 / len(I)
    plt.plot(q, I, c=colors[j], dashes=dashes[j], lw=lw[j], zorder=j,
             label=r'ffsas nit=%d, $\chi^2$=%.2f (%d pnts)' % 
             (results['saved_res'][step]['nit'], chi2, len(I)))
    
# MC-SAS
data_mc = np.loadtxt('PS3_data/mcsas_I(q).dat', skiprows=1)
q_mc = data_mc[:, 0] / 1e10
I_mc = data_mc[:, 3]
I_mc_mu = data_mc[:, 1]
I_mc_sigma = data_mc[:, 2]
chi2_mc = np.linalg.norm((I_mc_mu - I_mc) / I_mc_sigma) ** 2 / len(I_mc)
plt.plot(q_mc, I_mc, c=colors[j + 1], dashes=dashes[j + 1], lw=lw[j + 1],
         label=r'MC-SAS, $\chi^2$=%.2f (%d pnts)' % (chi2_mc, len(I_mc)))
plt.axvline(q_mc[-1], lw=1., c=colors[j + 1], ls='-')
plt.text(q_mc[-1], 10**4.2, 'MC-SAS limit', color=colors[j + 1], rotation=90, 
         ha='right', va='top', fontsize=10)

# SASView
data_sv = np.loadtxt('PS3_data/sasview_I(q).dat', skiprows=1)
q_sv = data_sv[:, 0]
I_sv = data_sv[:, 1]
chi2_sv = np.linalg.norm((mu[:len(I_sv)] - I_sv) / sigma[:len(I_sv)]) ** 2 / len(I_sv)
plt.plot(q_sv, I_sv, c=colors[j + 2], dashes=dashes[j + 2], lw=lw[j + 2],
         label=r'SASView, $\chi^2$=%.2f (%d pnts)' % (chi2_sv, len(I_sv)))
plt.axvline(q_sv[-1], lw=1., c=colors[j + 2], ls='-')
plt.text(q_sv[-1], 10**4.2, 'SASView limit', color=colors[j + 2], 
         rotation=90, ha='right', va='top', fontsize=10)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ ($\AA^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
handles, labels = plt.gca().get_legend_handles_labels()
handles.insert(0, handles.pop())
labels.insert(0, labels.pop())
plt.legend(handles, labels, loc='lower left')
plt.ylim((1e-4, 10 ** 4.5))
plt.xlim(q[0] / 1.1, 1e-1)
plt.title('Intensity fit for dataset PS3')
plt.savefig(output_dir / 'I(q).png', bbox_inches='tight', facecolor='w')
plt.show()


# radius distribution
plt.figure(dpi=200, figsize=(8, 4))
for j, step in enumerate(plot_steps):
    w = results['saved_res'][step]['w_dict']['r']
    plt.plot(r, w, c=colors[j], dashes=dashes[j], lw=lw[j], zorder=-j,
             label=r'$w(r)$, ffsas nit=%d, npts=%d, wct=%.1f sec' % 
             (results['saved_res'][step]['nit'], len(r),
              results['saved_res'][step]['wct']))


# MC-SAS
rw_mc = np.loadtxt('PS3_data/mcsas_w(r).dat', skiprows=1)
r_mc = rw_mc[:, 0] * 1e10
w_mc = rw_mc[:, 2] / r_mc ** 3
w_mc_interp = interpolate.interp1d(r_mc, w_mc)(r)
w_mc_interp /= np.sum(w_mc_interp)
plt.plot(r, w_mc_interp, c=colors[j + 1], dashes=dashes[j + 1], lw=lw[j + 1],
         label=r'$w(r)$, MC-SAS, npts=%d, wct=7.7 hr' % (len(r_mc)))

# # SASView
rw_sv = np.loadtxt('PS3_data/sasview_w(r).dat', skiprows=1)
r_sv = rw_sv[:, 0]
w_sv = rw_sv[:, 1]
w_sv_interp = interpolate.interp1d(r_sv, w_sv, fill_value="extrapolate")(r)
w_sv_interp /= np.sum(w_sv_interp)
plt.plot(r, w_sv_interp, c=colors[j+2], dashes=dashes[j + 2], lw=lw[j + 2],
         label=r'$w(r)$, SASView, npts=%s, wct<0.5 sec' % 'N/A')

plt.xlim(r[5], r[-5])
plt.ylabel(r'Weights, $w$')
plt.xlabel(r'Radius, $r$ ($\AA$)')
plt.legend(loc='upper left')
plt.title('Inverted radius distribution for dataset PS3')
plt.savefig(output_dir / 'w(r).png', bbox_inches='tight', facecolor='w')
plt.show()

---